In [ ]:
# path operations
from glob import glob
import os
from pathlib import Path

# data format and storage
from collections import namedtuple
import pickle

# numerical tools
import numpy as np
import scipy.stats
import pandas as pd

# plotting tools
from matplotlib import pyplot as plt
# %matplotlib notebook

# interactive notebook features
from tqdm import tqdm_notebook as tqdm
from ipywidgets import interact

# meg analysis
import mne

In [ ]:
bad_elecs = []

for pid in range(len(psds)):
    bads = np.zeros_like(all_parameters[pid].rsquared, dtype='bool')
    bads = bads | (all_parameters[pid].slopes > 0)
    bads = bads | (all_parameters[pid].intercepts >
                   all_parameters[pid].intercepts.mean() + 2 * all_parameters[pid].intercepts.std())
    bads = bads | (all_parameters[pid].intercepts <
                   all_parameters[pid].intercepts.mean() - 2 * all_parameters[pid].intercepts.std())
    bad_elecs.append(bads)
    print(bads.mean())

In [ ]:
pltdata = np.log10(np.stack([p[~bad_elecs[i], :].mean(axis=0)
                             for i, p in enumerate(psds)],
                             axis=-1))

x = freqs

plt.figure()
plt.fill_between(x,
                 np.mean(pltdata, axis=-1)-scipy.stats.sem(pltdata, axis=-1),
                 np.mean(pltdata, axis=-1)+scipy.stats.sem(pltdata, axis=-1))
# plt.plot(x, pltdata, color=(0.5, 0.5, 1, 0.5))
plt.plot(x, np.mean(pltdata, axis=-1), color='orange')
plt.show()

In [ ]:
ages = np.array([p.age for p in all_parameters])
young = ages < np.percentile(ages, 25)
old = ages > np.percentile(ages, 75)

In [ ]:
plt.figure(figsize=(12, 8))

legendentries = []
for group, color, label in [(young, [1, 0.4, 0.4], 'Youngest 25%'), (old, [0.4, 0.4, 1], 'Oldest 25%')]:
    pltdata = np.log10(np.stack([p[~bad_elecs[i], :].mean(axis=0)
                                 for i, p in enumerate(psds)
                                 if group[i]],
                                axis=-1))
    plt.fill_between(
        x, np.mean(pltdata, axis=-1)-scipy.stats.sem(pltdata, axis=-1),
        np.mean(pltdata, axis=-1)+scipy.stats.sem(pltdata, axis=-1),
        color=color+[0.3]
    )
    legendentries.append(plt.plot(x, np.mean(pltdata, axis=-1), color=color, label=label))

plt.ylabel('Power ($log_{10}(mV^2)$)', fontsize=20)
plt.xlabel('Frequency (Hz)', fontsize=20)
plt.legend(fontsize=16)
plt.show()

In [ ]:
def individual_spectrum(subjectno):
    dat = psds[subjectno]
    plt.figure()
    # plot individual lines, faint
    plt.plot(freqs, np.log10(dat.T), color=(0.3, 0.3, 1.0, 0.2))
    plt.plot(freqs, np.log10(dat).mean(axis=0), color=(1, 0.3, 0.3))
    plt.show()

interact(individual_spectrum, subjectno=(0, len(psds)-1))

In [ ]:
y = np.array([p.slopes for p in all_parameters])

plt.figure()
plt.boxplot(y.T)
plt.title('Regression slopes')
plt.show()

y = np.array([p.rsquared for p in all_parameters])

plt.figure()
plt.boxplot(y.T)
plt.title('$r^2$ values')
plt.show()


In [ ]:
x = [p.age for p in all_parameters]
y = [np.mean(p.slopes[~bad_elecs[i]]) for i, p in enumerate(all_parameters)]

plt.figure(figsize=(12, 8))
plt.scatter(x, y, c=x)
plt.xlabel(r'Age (Years)', fontsize=20)
plt.ylabel(r'Regression slope ($log_{10}(mV^2) \times Hz^{-1}$)', fontsize=20)
plt.show()

print(scipy.stats.pearsonr(x, y))


In [ ]:
# Load the short-term memory data
stm = pd.read_csv(
    camcanroot / 'cc700-scored' / 'VSTMcolour' / 'release001' / 'summary' / 'VSTMcolour_summary.txt',
    sep='\t', skiprows=8, index_col=0
)

In [ ]:
ages = [p.age for p in all_parameters if p.pid in stm.index]
slopes = [np.mean(p.slopes[~bad_elecs[i]]) for i, p in enumerate(all_parameters) if p.pid in stm.index]

x = slopes
y = [float(stm.loc[p.pid, 'Prcsn_ss3'])
                 for p in all_parameters
                 if p.pid in stm.index]

print(scipy.stats.pearsonr(x, y))

plt.figure(figsize=(12, 8))
plt.scatter(x, y)
plt.ylabel(r'Short term memory performance', fontsize=20)
plt.xlabel(r'Regression slope ($log_{10}(mV^2) \times Hz^{-1}$)', fontsize=20)
plt.show()